... intro stuff

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import glob
import hashlib
import json
import os
import re
import struct
import sys
import tarfile
import time

import numpy as np
from six.moves import urllib
import tensorflow as tf
from tensorflow.contrib.learn import ModeKeys

from tensorflow.python.platform import gfile
from tensorflow.python.util import compat


# If you've already downloaded the inception model, and it's elsewhere, 
# edit this path to reflect that so you don't need to re-download.
INCEPTION_MODEL_DIR = '/tmp/imagenet'
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
BOTTLENECK_TENSOR_NAME = 'pool_3/_reshape:0'
BOTTLENECK_TENSOR_SIZE = 2048
MODEL_INPUT_WIDTH = 299
MODEL_INPUT_HEIGHT = 299
MODEL_INPUT_DEPTH = 3
JPEG_DATA_TENSOR_NAME = 'DecodeJpeg/contents:0'
RESIZED_INPUT_TENSOR_NAME = 'ResizeBilinear:0'

# Edit IMAGE_PATH_LIST to specify the list of images that you'd like to 
# categorize against your trained model
IMAGE_PATH_LIST = ['hugs_photos/hugs/9783035421_d4c6569fda.jpg',
      'hugs_photos/hugs/9160821720_d1d926be43_b.jpg',
      'hugs_photos/not-hugs/8855807326_18ab8a583e.jpg',
      'hugs_photos/not-hugs/8599546166_b3c0fd6495.jpg']


...

In [ ]:
# Replace MODEL_DIR with the path to the directory in which your learned model resides.
MODEL_DIR = '/tmp/tfmodels/img_classify/1477497910'

...

In [2]:
def maybe_download_and_extract():
  """Download and extract the inception model tar file.
  If the pretrained model we're using doesn't already exist, this function
  downloads it from the TensorFlow.org website and unpacks it into a directory.
  """
  dest_directory = INCEPTION_MODEL_DIR
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(dest_directory, filename)
  if not os.path.exists(filepath):

    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' %
                       (filename,
                        float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()

    filepath, _ = urllib.request.urlretrieve(DATA_URL,
                                             filepath,
                                             _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(dest_directory)


def create_inception_graph():
  """"Creates a graph from saved GraphDef file and returns a Graph object.
  """
  with tf.Session() as sess:
    model_filename = os.path.join(
        INCEPTION_MODEL_DIR, 'classify_image_graph_def.pb')
    with gfile.FastGFile(model_filename, 'rb') as f:
      graph_def = tf.GraphDef()
      graph_def.ParseFromString(f.read())
      bottleneck_tensor, jpeg_data_tensor, resized_input_tensor = (
          tf.import_graph_def(graph_def, name='', return_elements=[
              BOTTLENECK_TENSOR_NAME, JPEG_DATA_TENSOR_NAME,
              RESIZED_INPUT_TENSOR_NAME]))
  return sess.graph, bottleneck_tensor, jpeg_data_tensor, resized_input_tensor


def run_bottleneck_on_image(sess, image_data, image_data_tensor,
                            bottleneck_tensor):
  """Runs inference on an image to extract the 'bottleneck' summary layer.
  """
  bottleneck_values = sess.run(
      bottleneck_tensor,
      {image_data_tensor: image_data})
  bottleneck_values = np.squeeze(bottleneck_values)
  return bottleneck_values

...

In [3]:
def make_image_predictions(
    classifier, jpeg_data_tensor, bottleneck_tensor, path_list, labels_list):
  """Use the learned model to make predictions."""

  if not labels_list:
    output_labels_file = os.path.join(MODEL_DIR, "output_labels.json")
    if gfile.Exists(output_labels_file):
      with open(output_labels_file, 'r') as lfile:
        labels_string = lfile.read()
        labels_list = json.loads(labels_string)
        print("labels list: %s" % labels_list)
    else:
      print("Labels list %s not found" % output_labels_file)
      return None

  sess = tf.Session()
  bottlenecks = []
  print("Predicting for images: %s" % path_list)
  for img_path in path_list:
    # get bottleneck for an image path. Don't cache the bottleneck values here.
    if not gfile.Exists(img_path):
      tf.logging.fatal('File does not exist %s', img_path)
    image_data = gfile.FastGFile(img_path, 'rb').read()
    bottleneck_values = run_bottleneck_on_image(sess, image_data,
                                                jpeg_data_tensor,
                                                bottleneck_tensor)
    bottlenecks.append(bottleneck_values)
  prediction_input = np.array(bottlenecks)
  predictions = classifier.predict(x=prediction_input, as_iterable=True)
  print("Predictions:")
  for _, p in enumerate(predictions):
    print(p["class"])
    print(p["index"])
    print(labels_list[p["index"]])

...

In [4]:
def add_final_training_ops(
    class_count, mode, final_tensor_name,
    bottleneck_input, ground_truth_input):
  """Adds a new softmax and fully-connected layer for training.
  """

  # Organizing the following ops as `final_training_ops` so they're easier
  # to see in TensorBoard
  train_step = None
  cross_entropy_mean = None

  layer_name = 'final_training_ops'
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      layer_weights = tf.Variable(
          tf.truncated_normal(
              [BOTTLENECK_TENSOR_SIZE, class_count],
              stddev=0.001), name='final_weights')
      # variable_summaries(layer_weights, layer_name + '/weights')
    with tf.name_scope('biases'):
      layer_biases = tf.Variable(tf.zeros([class_count]), name='final_biases')
      # variable_summaries(layer_biases, layer_name + '/biases')
    with tf.name_scope('Wx_plus_b'):
      logits = tf.matmul(bottleneck_input, layer_weights) + layer_biases
      tf.histogram_summary(layer_name + '/pre_activations', logits)

  final_tensor = tf.nn.softmax(logits, name=final_tensor_name)
  tf.histogram_summary(final_tensor_name + '/activations', final_tensor)

  if mode in [ModeKeys.EVAL, ModeKeys.TRAIN]:
    with tf.name_scope('cross_entropy'):
      cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
          logits, ground_truth_input)
      with tf.name_scope('total'):
        cross_entropy_mean = tf.reduce_mean(cross_entropy)
      tf.scalar_summary('cross entropy', cross_entropy_mean)

    with tf.name_scope('train'):
      train_step = tf.train.GradientDescentOptimizer(
          ARGFLAGS.learning_rate).minimize(
              cross_entropy_mean,
              global_step=tf.contrib.framework.get_global_step())

  return (train_step, cross_entropy_mean, final_tensor)


def add_evaluation_step(result_tensor, ground_truth_tensor):
  """Inserts the operations we need to evaluate the accuracy of our results.
  """
  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(result_tensor, 1), \
                                    tf.argmax(ground_truth_tensor, 1))
    with tf.name_scope('accuracy'):
      evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.scalar_summary('accuracy', evaluation_step)
  return evaluation_step


def make_model_fn(class_count):

  def _make_model(bottleneck_input, ground_truth_input, mode, params):

    prediction_dict = {}
    train_step = None
    cross_entropy = None

    # Add the new layer that we'll be training.
    (train_step, cross_entropy,
     final_tensor) = add_final_training_ops(
        class_count, mode, 'final_result',
        bottleneck_input, ground_truth_input)

    if mode in [ModeKeys.EVAL, ModeKeys.TRAIN]:
      # Create the operations we need to evaluate accuracy
      add_evaluation_step(final_tensor, ground_truth_input)

    if mode == ModeKeys.INFER:
      predclass = tf.argmax(final_tensor, 1)
      prediction_dict = {"class": final_tensor, "index": predclass}

    return prediction_dict, cross_entropy, train_step

  return _make_model

....

In [5]:
# load the labels list, needed to create the model; if it's 
# not there, we can't proceed
output_labels_file = os.path.join(MODEL_DIR, "output_labels.json")
if gfile.Exists(output_labels_file):
  with open(output_labels_file, 'r') as lfile:
    labels_string = lfile.read()
    labels_list = json.loads(labels_string)
    print("labels list: %s" % labels_list)
    class_count = len(labels_list)
else:
  print("Labels list %s not found; we can't proceed." % output_labels_file)


labels list: [u'not hugs', u'hugs']


....

In [6]:
# Set up the pre-trained graph 
maybe_download_and_extract()
graph, bottleneck_tensor, jpeg_data_tensor, resized_image_tensor = (
    create_inception_graph())
print("Finished.")

Finished.


...

In [7]:
# Define the custom estimator
model_fn = make_model_fn(class_count)
model_params = {}
classifier = tf.contrib.learn.Estimator(
    model_fn=model_fn, params=model_params, model_dir=MODEL_DIR)

...

In [8]:
make_image_predictions(
    classifier, jpeg_data_tensor, bottleneck_tensor, IMAGE_PATH_LIST, labels_list)

Predicting for images: ['hugs_photos/hugs/9783035421_d4c6569fda.jpg', 'hugs_photos/hugs/9160821720_d1d926be43_b.jpg', 'hugs_photos/not-hugs/8855807326_18ab8a583e.jpg', 'hugs_photos/not-hugs/8599546166_b3c0fd6495.jpg']
Predictions:
[  3.61323851e-04   9.99638677e-01]
1
hugs
[ 0.00209123  0.99790883]
1
hugs
[ 0.99897695  0.00102306]
0
not hugs
[  9.99771416e-01   2.28555917e-04]
0
not hugs


...